In [1]:
%run sun.py

import serial
import time

arduino = serial.Serial(port='COM4', baudrate=115200, timeout=.1)

def R_opt(beta_ax, az_ax, el, az,limit=90):
    beta_ax = np.radians(beta_ax)
    az_ax = np.radians(az_ax)
    
    zen = np.radians(90 - el)
    
    azrad = np.radians(az)
        
    arg = np.sin(zen)*np.sin(azrad-az_ax)/ \
            (np.sin(zen)*np.cos(azrad-az_ax)*np.sin(beta_ax) \
             + np.cos(zen)*np.cos(beta_ax))
    
    phi = np.where((arg < 0) & ((azrad-az_ax) > 0) , 180, 
            np.where((arg > 0) & ((azrad-az_ax) < 0), -180,0))
    
    
    R = np.degrees(np.arctan(arg)) + phi
    
    if R>90:
        R = limit
    if R<-90:
        R = -limit
    
    return R

def inc_SA(R, beta_ax, az_ax, el, az): 
    R = np.radians(R)
    beta_ax = np.radians(beta_ax)
    az_ax = np.radians(az_ax)
    el = np.radians(90-el)
    az = np.radians(az)
    
    arg = np.cos(R)*(np.sin(el)*np.cos(az-az_ax)*np.sin(beta_ax) \
                    +np.cos(el)*np.cos(beta_ax)) + \
                    np.sin(R)*np.sin(el)*np.sin(az-az_ax)
    return np.degrees(np.arccos(arg))


time_zone = 4
year = 2023
month = 1
day = 1
hr = 8 + time_zone
mn = 9
s = 15
lat = 43.5
long = -80.5

pos = sunPosition(year,month,day,hr,mn,s,lat,long)

beta_ax = 20
az_ax = 180
el = pos[1]
az = pos[0]

R = R_opt(beta_ax, az_ax, el, az,limit=90)

#angle = inc_SA(R, beta_ax, az_ax, el, az)

#print(angle)

if R == 90 & hr < 5:
    R = -90

print(R)

arduino.write(bytes(angle, 'utf-8'))
time.sleep(0.05)


-89.70956662287676
